In [1]:
###### https://www.kaggle.com/c/text-normalization-challenge-english-language


import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('train.csv')
df.head()

,sentence_id,token_id,class,before,after,poc
0,0,0,PLAIN,Brillantaisia,Brillantaisia,NN
1,0,1,PLAIN,is,is,VBZ
2,0,2,PLAIN,a,a,DT
3,0,3,PLAIN,genus,genus,NN
4,0,4,PLAIN,of,of,IN


In [3]:
df.isnull().sum()

sentence_id     0
token_id        0
class           0
before         74
after          69
poc             0
dtype: int64

In [4]:
df = df.fillna(method='ffill')

In [5]:
df.isnull().sum()

sentence_id    0
token_id       0
class          0
before         0
after          0
poc            0
dtype: int64

In [6]:
df['sentence_id'].nunique(), df.before.nunique(), df['class'].nunique()

(748066, 486442, 16)

In [7]:
df.groupby('class').size().reset_index(name='counts')

,class,counts
0,ADDRESS,522
1,CARDINAL,133744
2,DATE,258348
3,DECIMAL,9821
4,DIGIT,5442
5,ELECTRONIC,5162
6,FRACTION,1196
7,LETTERS,152795
8,MEASURE,14783
9,MONEY,6128


In [8]:
X = df.drop(['sentence_id','class','after','token_id'], axis=1)
X.head()


,before,poc
0,Brillantaisia,NN
1,is,VBZ
2,a,DT
3,genus,NN
4,of,IN


In [9]:
X.columns

Index(['before', 'poc'], dtype='object')

In [10]:
v = DictVectorizer()

In [11]:
X = v.fit_transform(X.to_dict('records'))


In [12]:
X.shape

(9918441, 486485)

In [13]:
y = df['class'].values

In [14]:
classes = np.unique(y)

In [15]:
classes = classes.tolist()
classes

['ADDRESS',
 'CARDINAL',
 'DATE',
 'DECIMAL',
 'DIGIT',
 'ELECTRONIC',
 'FRACTION',
 'LETTERS',
 'MEASURE',
 'MONEY',
 'ORDINAL',
 'PLAIN',
 'PUNCT',
 'TELEPHONE',
 'TIME',
 'VERBATIM']

In [16]:
X.shape, y.shape

((9918441, 486485), (9918441,))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)

In [18]:
X_train.shape, y_train.shape

((6645355, 486485), (6645355,))

In [19]:
X_test.shape, y_test.shape

((3273086, 486485), (3273086,))

In [20]:

new_classes = classes.copy()
new_classes.pop()
new_classes

['ADDRESS',
 'CARDINAL',
 'DATE',
 'DECIMAL',
 'DIGIT',
 'ELECTRONIC',
 'FRACTION',
 'LETTERS',
 'MEASURE',
 'MONEY',
 'ORDINAL',
 'PLAIN',
 'PUNCT',
 'TELEPHONE',
 'TIME']

In [ ]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

In [ ]:

print(classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=new_classes))

In [ ]:
sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train, classes)

In [ ]:
print(classification_report(y_pred=sgd.predict(X_test), y_true=y_test, labels=new_classes))

In [ ]:

nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

In [ ]:
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels = new_classes))

In [21]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [22]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['before'].values.tolist(), 
                                                           s['poc'].values.tolist(), 
                                                           s['class'].values.tolist())]
        self.grouped = self.data.groupby('sentence_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [23]:
getter = SentenceGetter(df)


In [24]:
sent = getter.get_next()
print(sent)

None


In [25]:
sentences = getter.sentences


In [26]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [27]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)


In [29]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=50,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=50,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [30]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=new_classes)

0.99666269705738553

In [31]:
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

             precision    recall  f1-score   support

    ADDRESS       0.97      0.42      0.59       154
   CARDINAL       0.97      0.97      0.97     44148
       DATE       0.98      0.99      0.99     85286
    DECIMAL       0.94      0.92      0.93      3280
      DIGIT       0.77      0.61      0.68      1719
 ELECTRONIC       0.96      0.78      0.86      1623
   FRACTION       0.88      0.68      0.77       410
    LETTERS       0.96      0.93      0.94     50714
    MEASURE       1.00      0.99      0.99      4902
      MONEY       0.99      0.91      0.95      2114
    ORDINAL       0.99      0.97      0.98      4220
      PLAIN       1.00      1.00      1.00   2426119
      PUNCT       1.00      1.00      1.00    620326
  TELEPHONE       0.94      0.84      0.88      1377
       TIME       0.96      0.88      0.92       465

avg / total       1.00      1.00      1.00   3246857



In [ ]:
X_train[0]

In [ ]:
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.grid_search import RandomizedSearchCV

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=50,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=new_classes)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=25,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels=new_classes))

In [32]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
TIME   -> DECIMAL 2.537920
MEASURE -> MEASURE 2.089429
DECIMAL -> DECIMAL 1.992695
DIGIT  -> LETTERS 1.890921
DATE   -> ELECTRONIC 1.837079
PLAIN  -> PLAIN   1.689222
VERBATIM -> VERBATIM 1.644348
ELECTRONIC -> DATE    1.492422
TIME   -> CARDINAL 1.414490
DATE   -> TIME    1.414211
ELECTRONIC -> ELECTRONIC 1.370165
ADDRESS -> CARDINAL 1.323073
DIGIT  -> DECIMAL 1.281697
CARDINAL -> CARDINAL 1.267240
LETTERS -> DIGIT   1.138752
DATE   -> DIGIT   1.127022
FRACTION -> DATE    1.093341
FRACTION -> CARDINAL 0.965752
PLAIN  -> DATE    0.910143
PLAIN  -> MONEY   0.900885

Top unlikely transitions:
ORDINAL -> LETTERS -1.181543
MONEY  -> LETTERS -1.205115
CARDINAL -> TIME    -1.260073
VERBATIM -> ELECTRONIC -1.292514
VERBATIM -> LETTERS -1.343782
ORDINAL -> ELECTRONIC -1.345779
VERBATIM -> ADDRESS -1.348680
ADDRESS -> ELECTRONIC -1.393938
LETTERS -> DECIMAL -1.439153
ADDRESS -> TELEPHONE -1.585387
VERBATIM -> CARDINAL -1.587668
ORDINAL -> CARDINAL -1.608915
ADDRESS -> DI

In [34]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
12.905875 DATE     word[-2:]:0s
10.714175 LETTERS  word.lower():eds
9.847006 PLAIN    word.lower():one
9.245423 PLAIN    word.lower():three
8.599327 DIGIT    word.lower():000
8.077553 LETTERS  word.lower():als
8.070118 LETTERS  word.lower():mes
8.013081 PLAIN    word.lower():million
7.829573 PLAIN    word.lower():two
7.786055 PLAIN    word.lower():four
7.640103 ORDINAL  word[-3:]:0th
7.618228 LETTERS  word.lower():métis
7.512676 MEASURE  word[-2:]:0%
7.507092 PLAIN    word.lower():mid
7.403130 DECIMAL  word[-2:]:.9
7.357601 ORDINAL  word[-3:]:7th
7.292916 TIME     word[-3:]::00
7.216838 DATE     word[-3:]:0's
7.206331 LETTERS  word.lower():iasi
7.146557 LETTERS  word.lower():ses
7.120744 ELECTRONIC word[-3:]:.uk
7.099762 MEASURE  word[-2:]:5%
7.093452 DIGIT    -1:word.lower():mid
7.076363 LETTERS  word.lower():tra
7.062820 MEASURE  word[-2:]:7%
7.000630 MEASURE  word[-2:]:9%
6.990388 ORDINAL  word[-3:]:5th
6.954561 MEASURE  word[-2:]:8%
6.946697 MEASURE  word[-2:]: m
6.92

In [40]:
import eli5

eli5.show_weights(crf, top=50)

From \ To,ADDRESS,CARDINAL,DATE,DECIMAL,DIGIT,ELECTRONIC,FRACTION,LETTERS,MEASURE,MONEY,ORDINAL,PLAIN,PUNCT,TELEPHONE,TIME,VERBATIM
ADDRESS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.289,-0.128,0.0,0.0,0.0
CARDINAL,0.0,0.129,-0.118,0.0,0.0,0.0,0.0,0.004,0.0,0.0,0.0,0.229,0.194,0.0,0.0,0.168
DATE,0.0,-0.061,-0.237,0.0,0.0,0.0,0.0,-0.127,0.0,0.0,0.0,0.199,0.282,0.0,0.0,-0.111
DECIMAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.204,0.0,0.0,0.0,0.0
DIGIT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.723,0.0,0.0,0.0,-0.232,-0.026,0.0,0.0,0.0
ELECTRONIC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.082,0.306,0.0,0.0,0.0
FRACTION,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.256,-0.039,0.0,0.0,0.0
LETTERS,0.0,0.174,-0.265,-0.003,0.711,0.0,0.0,-0.704,-0.016,0.0,-0.0,0.5,-0.093,0.731,0.0,-1.05
MEASURE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.268,0.0,0.0,-0.002
MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013,0.115,0.0,0.0,0.0


In [38]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=200,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)
eli5.show_weights(crf, top=10)

From \ To,ADDRESS,CARDINAL,DATE,DECIMAL,DIGIT,ELECTRONIC,FRACTION,LETTERS,MEASURE,MONEY,ORDINAL,PLAIN,PUNCT,TELEPHONE,TIME,VERBATIM
ADDRESS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.289,-0.128,0.0,0.0,0.0
CARDINAL,0.0,0.129,-0.118,0.0,0.0,0.0,0.0,0.004,0.0,0.0,0.0,0.229,0.194,0.0,0.0,0.168
DATE,0.0,-0.061,-0.237,0.0,0.0,0.0,0.0,-0.127,0.0,0.0,0.0,0.199,0.282,0.0,0.0,-0.111
DECIMAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.204,0.0,0.0,0.0,0.0
DIGIT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.723,0.0,0.0,0.0,-0.232,-0.026,0.0,0.0,0.0
ELECTRONIC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.082,0.306,0.0,0.0,0.0
FRACTION,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.256,-0.039,0.0,0.0,0.0
LETTERS,0.0,0.174,-0.265,-0.003,0.711,0.0,0.0,-0.704,-0.016,0.0,-0.0,0.5,-0.093,0.731,0.0,-1.05
MEASURE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.268,0.0,0.0,-0.002
MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013,0.115,0.0,0.0,0.0


In [41]:
eli5.show_weights(crf, top=10, feature_re='^word\.is',
                  horizontal_layout=True, show=['targets'])

In [43]:
len(crf.state_features_)

2187